<a href="https://colab.research.google.com/github/Murad460/Day_to_day_use_projs/blob/main/Decoding_Fingerprint_Using_Scanning_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Decoding Fingerprint Using Scanning Algorithm

# function camera():
#     finger_images ← camera.click()
#     return finger_images

# function decoder_logic():
#     finger_images ← camera()

#     common_pattern ← extract_common_fingerprint_points()

#     finger_cp ← pattern(common_pattern)

#     clean_image ← clean(finger_images)
#     enhanced_image ← clean_image.add(finger_cp)

#     return enhanced_image

# function pattern(img):
#     p ← AI_model.predict(img)
#     return p

# function generate():
#     data ← decoder_logic()
#     model_3D ← generate_3D_model(data)
#     print(model_3D)

# function generate_3D_model():
#     model ← image.add(other_images)   // merge images in 3D space
#     return model

# function threeD_model():
#     img ← original_image
#     axis_angle ← 360 - original_image.axis

#     for each img_i in other_images:
#         rotate(img_i, 360 - i)

#     pixel ← 0
#     for each i in img:
#         pixel ← pixel + i

#     return decode_image(pixel)


In [ ]:
!pip install opencv-python numpy matplotlib scikit-image
# python fingerprint_decoder_realtime.py


In [1]:
import cv2
import numpy as np
import time
from pathlib import Path
from dataclasses import dataclass
from typing import Tuple, List, Dict

# ===================== DATA STRUCTURE =====================

@dataclass
class FingerprintDetection:
    bbox: Tuple[int, int, int, int]
    fingerprint_image: np.ndarray
    enhanced_image: np.ndarray
    ridge_map: np.ndarray
    confidence: float
    quality_metrics: Dict


# ===================== FINGER DETECTOR =====================

class FingerDetector:
    def __init__(self):
        self.min_area = 2000
        self.max_area = 50000

    def detect_skin_fingers(self, frame: np.ndarray) -> List[Tuple[int, int, int, int]]:
        ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
        lower = np.array([0, 133, 77], dtype=np.uint8)
        upper = np.array([255, 173, 127], dtype=np.uint8)
        mask = cv2.inRange(ycrcb, lower, upper)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        regions = []
        for c in contours:
            area = cv2.contourArea(c)
            if self.min_area < area < self.max_area:
                x, y, w, h = cv2.boundingRect(c)
                if h / float(w + 1e-6) > 1.2:
                    regions.append((x, y, w, h))
        return regions


# ===================== ENHANCER =====================

class FingerprintEnhancer:
    def __init__(self):
        self.clahe = cv2.createCLAHE(2.0, (8, 8))

    def enhance(self, img: np.ndarray) -> np.ndarray:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        denoise = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
        clahe = self.clahe.apply(denoise)
        blur = cv2.GaussianBlur(clahe, (0, 0), 3)
        sharp = cv2.addWeighted(clahe, 1.5, blur, -0.5, 0)
        return sharp

    def extract_ridges(self, img: np.ndarray) -> np.ndarray:
        enhanced = self.enhance(img)
        edges = cv2.Canny(enhanced, 50, 150)
        return edges

    def quality(self, img: np.ndarray) -> Dict:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lap = cv2.Laplacian(gray, cv2.CV_64F)
        score = min(1.0, np.var(lap) / 500)
        return {"quality_score": score}


# ===================== 3D RECONSTRUCTION =====================

class Fingerprint3DReconstructor:
    def __init__(self, out_dir="fingerprint_3d", target_size=(200, 250)):
        self.images = []
        self.target_size = target_size
        self.out = Path(out_dir)
        self.out.mkdir(exist_ok=True)

    def add(self, enhanced: np.ndarray):
        resized = cv2.resize(enhanced, self.target_size)
        self.images.append(resized)

    def can_generate(self):
        return len(self.images) >= 10

    def generate(self):
        if len(self.images) < 2:
            raise ValueError("Not enough images")

        stack = np.stack(self.images, axis=0)
        avg = np.mean(stack, axis=0)

        sobelx = cv2.Sobel(avg, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(avg, cv2.CV_64F, 0, 1, ksize=3)

        depth = np.sqrt(sobelx**2 + sobely**2)
        depth = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX)
        depth = depth.astype(np.uint8)

        fname = self.out / f"3d_depth_{int(time.time())}.png"
        cv2.imwrite(str(fname), depth)

        print(f"[✓] 3D model saved: {fname}")
        return depth



# ===================== REALTIME SYSTEM =====================

class RealtimeFingerprintSystem:
    def __init__(self):
        self.detector = FingerDetector()
        self.enhancer = FingerprintEnhancer()
        self.reconstructor = Fingerprint3DReconstructor()
        self.cap = cv2.VideoCapture(0)
        self.saved = 0

    def run(self):
        if not self.cap.isOpened():
            print("❌ Camera not available")
            return

        print("▶ Fingerprint system running (press q to quit, s to save)")

        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            detections = self.detector.detect_skin_fingers(frame)

            for (x, y, w, h) in detections:
                roi = frame[y:y+h, x:x+w]
                if roi.size == 0:
                    continue

                enhanced = self.enhancer.enhance(roi)
                ridges = self.enhancer.extract_ridges(roi)
                quality = self.enhancer.quality(roi)

                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"Q:{quality['quality_score']:.2f}",
                            (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

                cv2.imshow("Enhanced", enhanced)
                cv2.imshow("Ridges", ridges)

            cv2.imshow("Fingerprint Detection", frame)

            key = cv2.waitKey(100) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s') and detections:
                self.reconstructor.add(enhanced)
                self.saved += 1
                print(f"[+] Saved image {self.saved}/10")

                if self.reconstructor.can_generate():
                    depth = self.reconstructor.generate()
                    cv2.imshow("3D Depth Map", depth)
                    self.reconstructor.images.clear()
                    self.saved = 0

        self.cap.release()
        cv2.destroyAllWindows()


# ===================== MAIN =====================q

if __name__ == "__main__":
    system = RealtimeFingerprintSystem()
    system.run()


❌ Camera not available
